In [13]:
import numpy as np
from lookup_table import CaseNum2EdgeOffset, getCaseNum
import trimesh
import os
import time

In [14]:
def marching_cube(thres,cells):
    # vertices use dictionary to avoid duplicate axes
    vertex_array = {}
    face_array = []
    t1 = time.time()
    # -------------------TODO------------------ 
    # compute vertices and faces
    # vertices: [N, 3]
    # faces: [M, 3], e.g. np.array([[0,1,2]]) means a triangle composed of vertices[0], vertices[1] and vertices[2]
    # for-loop is allowed to reduce difficulty
    # -------------------TODO------------------ 
    
    def interp(p1, p2, v1, v2):
        return (thres - v1) / (v2 - v1) * (p2 - p1) + p1
    
    edges = {}
    vcnt = 0
    for i in range(cells.shape[0] - 1):
        for j in range(cells.shape[1] - 1):
            for k in range(cells.shape[2] - 1):
                case_nums = getCaseNum(i, j, k, thres, cells)
                faces = []
                for case_num in case_nums:
                    if case_num == -1:
                        break
                    edge_offset = CaseNum2EdgeOffset[case_num]
                    p1 = np.array([i, j, k]) + edge_offset[:3]
                    p2 = np.array([i, j, k]) + edge_offset[3:]
                    v1 = cells[p1[0], p1[1], p1[2]]
                    v2 = cells[p2[0], p2[1], p2[2]]
                    p = interp(p1, p2, v1, v2)
                    key = (p1[0], p1[1], p1[2], p2[0], p2[1], p2[2])
                    if edges.get(key) is None:
                        edges[key] = vcnt
                        vertex_array[vcnt] = p
                        vcnt += 1
                    faces.append(edges[key])
                    if len(faces) == 3:
                        face_array.append(np.array(faces))
                        faces = []
                    
    
    t2 = time.time()
    print("\nTime taken by algorithm\n"+'-'*40+"\n{} s".format(t2-t1))
    vertex_array = list(vertex_array.values())
    return np.array(vertex_array), np.array(face_array)

In [15]:
# reconstruct these two animals
shape_name_lst = ['spot', 'bob']
for shape_name in shape_name_lst:
    data = np.load(os.path.join('data', shape_name + '_cell.npy'))
    verts, faces = marching_cube(0, data)
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    mesh_txt = trimesh.exchange.obj.export_obj(mesh)
    with open(os.path.join('../results', shape_name + '.obj'),"w") as fp:
        fp.write(mesh_txt)


Time taken by algorithm
----------------------------------------
3.7642502784729004 s

Time taken by algorithm
----------------------------------------
3.874124526977539 s
